In [1]:
%reset -f
%matplotlib inline

In [2]:
import numpy as np
import scipy.io as sio
import sklearn as sk
from sklearn import decomposition

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import pandas as pd

import datetime

from scipy.io import savemat

In [3]:
cmap = sns.color_palette('RdBu',5)[:2] + sns.color_palette('RdBu',5)[-2:]
sns.set_palette(cmap)

In [4]:
def make_pairgrid(d):
    ''' in: (d1,d2,d3,d4) '''
    df = pd.DataFrame(np.concatenate(d))
    cond_labels = d[0].shape[0]*['fw1'] + d[1].shape[0]*['fw2'] + d[2].shape[0]*['bw1'] + d[3].shape[0]*['bw2']
    df['condition'] = cond_labels
    g = sns.PairGrid(df, hue='condition', diag_sharey=True)
    g.map_diag(plt.hist)
    g.map_offdiag(plt.plot)
    dmax = np.max(np.concatenate(d))
    g.add_legend()

In [5]:
monkey='D'

In [6]:
if monkey=='D':
    data = sio.loadmat('/Users/jeff/Documents/Python/_projects/cyclingRNN/drakeFeb.mat')
else:
    data = sio.loadmat('/Users/jeff/Documents/Python/_projects/cyclingRNN/cousFeb.mat')

In [7]:
m1 = data['D'][0,0]['M1']
emg = data['D'][0,0]['EMG']

m1 = np.reshape(m1, m1.shape[:2]+(4,)) # order = 'C' or 'F'
emg = np.reshape(emg, emg.shape[:2]+(4,)) # order = 'C' or 'F'

m1 = np.transpose(m1, [1,2,0])
emg = np.transpose(emg, [1,2,0])
# Preprocess data
# Normalize EMG
max_ = np.max(emg, axis=(0,1))
min_ = np.min(emg, axis=(0,1))
emg_ = (emg - min_)/(max_ - min_)

# Normalize M1
max_ = np.max(m1, axis=(0,1))
min_ = np.min(m1, axis=(0,1))
m1_ = m1/(max_ - min_ + 5)

# Select times + downsample 
times = np.arange(2000,4000, 25)
m1_ = m1[times]
emg_ = emg_[times]

In [8]:
from cycling_rnn import run_rnn

### TF stuff

In [ ]:
monkey='D'
beta1 = np.logspace(-3,0,7)
beta2 = np.logspace(-3,0,7)
learning_rate = 0.0003
num_iters=5000
load_prev=False
load_model_path='/Users/jeff/Documents/Python/_projects/cyclingRNN/saves/D_0713-1449-29'
local_machine=True

In [ ]:
y_tf = np.zeros((beta1.size, beta2.size), dtype=object)
x_tf = np.zeros((beta1.size, beta2.size), dtype=object)
loss_val = np.zeros((beta1.size, beta2.size), dtype=object)

for i, i_val in enumerate(beta1):
    for j, j_val in enumerate(beta2):
        print i,i_val
        print j,j_val
        y_tf[i,j], x_tf[i,j], loss_val[i,j] = run_rnn(monkey=monkey,
                                                        beta1=i_val,
                                                        beta2=j_val,
                                                        learning_rate=learning_rate,
                                                        num_iters=num_iters,
                                                        load_prev=load_prev,
                                                        load_model_path=load_model_path,
                                                        local_machine=local_machine)
        np.save('./y_tf', y_tf)
        np.save('./x_tf', x_tf)

0 0.001
0 0.001
Current run: D_0713-2255-42
iter: 0000 Loss: 0.183535
iter: 0500 Loss: 0.039971
iter: 1000 Loss: 0.027256
iter: 1500 Loss: 0.024666
iter: 2000 Loss: 0.024011
iter: 2500 Loss: 0.023792
iter: 3000 Loss: 0.023664
iter: 3500 Loss: 0.023562
iter: 4000 Loss: 0.023477
iter: 4500 Loss: 0.023404
iter: 4999 Loss: 0.023345
Finished
0 0.001
1 0.00316227766017
Current run: D_0713-2304-07
iter: 0000 Loss: 0.231415
iter: 0500 Loss: 0.047033
iter: 1000 Loss: 0.028165
iter: 1500 Loss: 0.025083
iter: 2000 Loss: 0.024436
iter: 2500 Loss: 0.024190
iter: 3000 Loss: 0.024016
iter: 3500 Loss: 0.023865
iter: 4000 Loss: 0.023731
iter: 4500 Loss: 0.023616
iter: 4999 Loss: 0.023523
Finished
0 0.001
2 0.01
Current run: D_0713-2312-30
iter: 0000 Loss: 0.351151
iter: 0500 Loss: 0.055913
iter: 1000 Loss: 0.028887
iter: 1500 Loss: 0.025555
iter: 2000 Loss: 0.024899
iter: 2500 Loss: 0.024602
iter: 3000 Loss: 0.024385
iter: 3500 Loss: 0.024194
iter: 4000 Loss: 0.024015
iter: 4500 Loss: 0.023857
iter: 49

## Plot fit quality

In [ ]:
ii = 0
jj = 0

In [ ]:
rows = 4
cols = 2

with sns.color_palette(n_colors=4):
    f, ax = plt.subplots(rows,cols, figsize=(20,20))
    for i in range(rows):
        for j in range(cols):
            muscle = np.ravel_multi_index((i,j), (rows,cols))
            ax[i,j].plot(emg_[:,:,muscle], linewidth=2, alpha=1)
            ax[i,j].plot(y_tf[ii,jj][:,:,muscle],'--',linewidth=4, alpha=0.5)
            ax[i,j].set_title(str(muscle))
plt.show()

## RNN neuron PC plots

In [ ]:
pca_x = sk.decomposition.PCA(n_components=5)
pca_y = sk.decomposition.PCA(n_components=5)

pca_x.fit(np.concatenate([x_tf[ii,jj][:,i,:] for i in range(4)]))
pca_y.fit(np.concatenate([y_tf[ii,jj][:,i,:] for i in range(4)]))

In [ ]:
# plot PCs for the RNN state variable
make_pairgrid([np.dot(x_tf[ii,jj][:,i,:], pca_x.components_.T) for i in range(4)])

## RNN activations

In [ ]:
rows = 10
cols = 4
f, ax = plt.subplots(rows,cols, figsize=(20,20))

for i in range(rows):
    for j in range(cols):
        neuron = np.ravel_multi_index((i,j), (rows,cols))
        ax[i,j].plot(x_tf[ii,jj][:,:,neuron])
        ax[i,j].set_title(str(neuron))
plt.show()

### export to stupid lab